In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import math
import random
import scipy.stats as stats

### 1. Data Overview

In [3]:
# Load data

data = pd.read_excel("/content/drive/MyDrive/Pacmann/P. Sampling/Keluarga Miskin Kota Yogyakarta, Kab. Sleman, dan Kab. Kulonprogo.xlsx")

In [4]:
# melihat 5 data teratas

data.head()

,Kelurahan,Kecamatan,Jumlah
0,Jogotirto,Berbah,410
1,Sendangtirto,Berbah,515
2,Tegaltirto,Berbah,370
3,Sumberrahayu,Moyudan,260
4,Sidokarto,Godean,490


In [5]:
#info Statistiknya

data.describe()

,Jumlah
count,219.000000
mean,352.442922
std,203.924906
min,10.000000
25%,206.000000
50%,327.000000
75%,500.500000
max,986.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Kelurahan  219 non-null    object
 1   Kecamatan  219 non-null    object
 2   Jumlah     219 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 5.3+ KB


Dataset terdiri 3 kolom:

- Kelurahan : Nama Kelurahan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo
- Kecamatan : Nama Kecamatan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo
- Jumlah : Jumlah keluarga miskin di setiap kelurahan

### 2. Memilih PSU

- Sejumlah 10 kecamatan akan diambil sebagai PSU
- Setiap Kelurahan memiliki jumlah keluarga miskin yang berbeda-beda

In [7]:
data['Kecamatan'].nunique()

43

In [8]:
data['Kelurahan'].nunique()

215

In [9]:
# Summary variables

N = 43                 # jumlah seluruh Kecamatan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo
n = 10                 # jumlah Kecamatan yang akan dijadikan sampel
pop_mean = np.mean(data['Jumlah'])

print(f"Jumlah Kecamatan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo                               : {N} kecamatan")
print(f"Jumlah Kecamatan yang akan diambil sebagai sampel                                                             : {n} kecamatan")
print(f'Rata-rata jumlah keluarga miskin per Kelurahan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo : {pop_mean:.0f} orang  ')

Jumlah Kecamatan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo                               : 43 kecamatan
Jumlah Kecamatan yang akan diambil sebagai sampel                                                             : 10 kecamatan
Rata-rata jumlah keluarga miskin per Kelurahan di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo : 352 orang  


In [10]:
# Random sampling 10 Kecamatan

unique_kecamatan = data.groupby('Kecamatan', group_keys=False).apply(lambda data: data.sample(1))
sample_kecamatan = unique_kecamatan['Kecamatan'].sample(n = 10, random_state = 3)
sample_kecamatan

186        Pengasih
123      Umbulharjo
34         Ngemplak
103       Ngampilan
102    Gedongtengen
97     Gondokusuman
171           Galur
61        Prambanan
44           Tempel
113          Kraton
Name: Kecamatan, dtype: object

In [11]:
# 10 Kecamatan yang terpilih secara random

kecamatan = data['Kecamatan']

jumlah_kelurahan = data[kecamatan.isin(['Pengasih',
 'Umbulharjo',
 'Ngemplak',
 'Ngampilan',
 'Gedongtengen',
 'Gondokusuman',
 'Galur',
 'Prambanan',
 'Tempel',
 'Kraton'])].groupby('Kecamatan')['Kelurahan'].count()\
.reset_index(name = 'jumlah_kelurahan')

jumlah_keluarga_miskin = data[kecamatan.isin(['Pengasih',
 'Umbulharjo',
 'Ngemplak',
 'Ngampilan',
 'Gedongtengen',
 'Gondokusuman',
 'Galur',
 'Prambanan',
 'Tempel',
 'Kraton'])].groupby('Kecamatan')['Jumlah'].sum()\
.reset_index(name = 'Jumlah')

kecamatan_terpilih = pd.merge(jumlah_kelurahan, jumlah_keluarga_miskin, on=['Kecamatan'])
kecamatan_terpilih

,Kecamatan,jumlah_kelurahan,Jumlah
0,Galur,7,849
1,Gedongtengen,2,701
2,Gondokusuman,5,1704
3,Kraton,3,1166
4,Ngampilan,2,567
5,Ngemplak,5,1531
6,Pengasih,7,3123
7,Prambanan,6,2435
8,Tempel,8,2414
9,Umbulharjo,7,1845


In [12]:
# Mencari nilai mu_r
# y_total adalah jumlah keluarga miskin dari Kecamatan yang dijadikan sample

y_total = np.sum(kecamatan_terpilih['Jumlah'])

# M adalah jumlah Kelurahan dari Kecamatan yang dijadikan sample
M = np.sum(kecamatan_terpilih['jumlah_kelurahan'])

mu_r = y_total / M

print(f'Total keluarga miskin dari seluruh Kecamatan yang ada di sampel          : {y_total} orang')
print(f'Total jumlah Kelurahan dari seluruh Kecamatan yang ada di sampel         : {M} kelurahan')
print(f'Estimasi rata-rata jumlah keluarga miskin                                : {mu_r:.0f} orang')

Total keluarga miskin dari seluruh Kecamatan yang ada di sampel          : 16335 orang
Total jumlah Kelurahan dari seluruh Kecamatan yang ada di sampel         : 52 kelurahan
Estimasi rata-rata jumlah keluarga miskin                                : 314 orang


### 3. Estimasi dengan 95% Confidence Interval

In [13]:
# menghitung sample varians terlebih dahulu

sc_square = np.sum((kecamatan_terpilih["Jumlah"] - mu_r*kecamatan_terpilih["jumlah_kelurahan"])**2) / (n-1)
print(f"Varians sampel                        : ({np.sqrt(sc_square):.0f})^2")

# selanjutnya menghitung estimator varians total populasi

var_y_total_est = N*(N-n)*(sc_square/n)
print(f'Varians dari estimator total populasi : ({np.sqrt(var_y_total_est):.0f}^2)')

Varians sampel                        : (596)^2
Varians dari estimator total populasi : (7095^2)


In [14]:
# mencari rata-rata Kelurahan dari setiap Kecamatan yang ada di sample

M_bar = np.mean(kecamatan_terpilih["jumlah_kelurahan"])
print(f"Rata-rata jumlah Kelurahan per Kecamatan                      : {M_bar:.0f}")


# hitung total dari populasi berdasarkan rata-rata

M_tot_est = N * M_bar
print(f"Estimasi jumlah Kelurahan dari Kecamatan yang ada di sample   : {M_tot_est:.0f}")


# hitung varians dari rata-rata

var_y_mean_est = var_y_total_est / (M_tot_est**2)
print(f"The variance of mean ssu. estimator                           : ({np.sqrt(var_y_mean_est):.0f})^2")

Rata-rata jumlah Kelurahan per Kecamatan                      : 5
Estimasi jumlah Kelurahan dari Kecamatan yang ada di sample   : 224
The variance of mean ssu. estimator                           : (32)^2


In [15]:
# menghitung z_stat pada 95% Confidence Interval

alpha = 0.05
z_stat = stats.norm.ppf(1 - alpha/2.)
print(f"z_stat              : {z_stat:.2f}")

# menghitung margin of error (d)

d = z_stat * np.sqrt(var_y_mean_est)
print(f"margin of error (d) : {d:.0f}")

z_stat              : 1.96
margin of error (d) : 62


In [16]:
# menghitung confidence interval

lb = mu_r - d
ub = mu_r + d

print(f"Dengan 95% confidence interval, nilai rata-rata asli jumlah keluarga miskin per kelurahan \n di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo berkisar antara {lb:.0f} sampai {ub:.0f} orang")

Dengan 95% confidence interval, nilai rata-rata asli jumlah keluarga miskin per kelurahan 
 di Kota Yogyakarta, Kabupaten Sleman, dan Kabupaten Kulonprogo berkisar antara 252 sampai 376 orang


### 3. Menghitung Jumlah Kecamatan yang Dibutuhkan Jika Ingin Mengurangi Margin of Error

In [17]:
# Margin of error yang baru
B = 5

D = ((B*M_bar)**2) / 4
print(D)

169.0


In [18]:
# Jumlah baru Kecamatan yang dibutuhkan

n_new = (N*sc_square) / (N*D + sc_square)
print(f'Untuk mendapatkan margin of error sebesar 5%, maka kita harus mengambil sampel sebanyak {n_new:.0f} Kecamatan')

Untuk mendapatkan margin of error sebesar 5%, maka kita harus mengambil sampel sebanyak 42 Kecamatan
